In [ ]:
import requests
import re
import os
import pandas as pd
import json
from tqdm.auto import tqdm
from functools import lru_cache

### Download data


In [ ]:
def get_settrade_symbols():
    r = requests.get("https://api.settrade.com/api/quote/symbols")
    assert r.status_code == 200
    r = r.json()
    symbols = [
        s["abbr_name"].lower()
        for s in r["symbols"]
        if s["product_type"] == "Stock"
        and "PUBLIC COMPANY LIMITED" in s["full_name_en"]
        and not re.match(".*-[FPQW]$|.*-[FPQW][0-9]$", s["abbr_name"])
    ]
    return symbols

In [ ]:
r = requests.get(
    "https://www.set.or.th/api/set/factsheet/cpall/price-performance?lang=en".format(
        symbol
    ),
    headers={
        "referer": "https://www.set.or.th/en/market/product/stock/quote/cpall/factsheet"
    },
)
r

In [ ]:
r.content

In [ ]:
@lru_cache(None)
def get_symbol_data(symbol):
    headers = {
        "referer": "https://www.set.or.th/en/market/product/stock/quote/{}/factsheet".format(
            symbol
        )
    }
    data = {}
    r = requests.get(
        "https://www.set.or.th/api/set/factsheet/{}/price-performance?lang=en".format(
            symbol
        ),
        headers=headers,
    )
    assert r.status_code == 200, symbol
    c = json.loads(r.content)["stock"]
    data.update(c)
    r = requests.get(
        "https://www.set.or.th/api/set/stock/{}/info?lang=en".format(symbol),
        headers=headers,
    )
    assert r.status_code == 200, symbol
    c = json.loads(r.content)
    data.update(c)
    r = requests.get(
        "https://www.set.or.th/api/set/factsheet/{}/highlight-data?lang=en".format(
            symbol
        ),
        headers=headers,
    )
    assert r.status_code == 200, symbol
    c = json.loads(r.content)
    data.update(c)
    r = requests.get(
        "https://www.set.or.th/api/set/factsheet/{}/financial-ratio?lang=en".format(
            symbol
        ),
        headers=headers,
    )
    assert r.status_code == 200, symbol
    c = json.loads(r.content)
    data.update(
        {ratio["accountName"]: ratio["value"] for ratio in c[0]["data"]},
        headers=headers,
    )
    return data

In [ ]:
symbols = get_settrade_symbols()
buffer = []
for symbol in tqdm(symbols):
    try:
        buffer.append(get_symbol_data(symbol))
    except:
        print("Error loading : " + symbol)
        pass

In [ ]:
df = pd.DataFrame(buffer)
df.to_csv("data.csv", index=False)